<a href="https://colab.research.google.com/github/AtharvBhat/PIAYN/blob/main/PIAYN_TextClassificationLRABaseline_New_2k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.5 MB 8.5 MB/s 
     |████████████████████████████████| 895 kB 52.4 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 


In [2]:
!pip install -q datasets

     |████████████████████████████████| 325 kB 8.5 MB/s 
     |████████████████████████████████| 212 kB 52.0 MB/s 
     |████████████████████████████████| 1.1 MB 34.2 MB/s 
     |████████████████████████████████| 136 kB 24.6 MB/s 
     |████████████████████████████████| 127 kB 55.3 MB/s 
     |████████████████████████████████| 271 kB 54.1 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
     |████████████████████████████████| 144 kB 58.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install ml_collections
!pip install absl-py

     |████████████████████████████████| 77 kB 4.4 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=e53e4be6611ad307dbf24877ad6ad8aebc5dbf067fd94770f917705567720295
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [4]:
!git clone https://github.com/google-research/long-range-arena.git
!git clone https://github.com/mlpen/Nystromformer.git
!git clone https://github.com/AtharvBhat/PIAYN.git

Cloning into 'long-range-arena'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 474 (delta 330), reused 423 (delta 282), pack-reused 0
Receiving objects: 100% (474/474), 153.28 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (330/330), done.
Cloning into 'Nystromformer'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 292 (delta 140), reused 147 (delta 42), pack-reused 0
Receiving objects: 100% (292/292), 10.82 MiB | 30.27 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Cloning into 'PIAYN'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 19 (delta 4), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (19/19), done.


## Set up output folders

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Set the task
task = 'text'

#Set Seq Length
max_length = 2000

In [7]:
import os

baseDir = "/content/drive/MyDrive"
piaynDir = os.path.join(baseDir, "PIAYN")

if not os.path.exists(piaynDir):
  os.makedirs(piaynDir)

piaynTaskDir = os.path.join(piaynDir, task)

if not os.path.exists(piaynTaskDir):
  os.makedirs(piaynTaskDir)

piaynTaskModelDir = os.path.join(piaynTaskDir, "models")

if not os.path.exists(piaynTaskModelDir):
  os.makedirs(piaynTaskModelDir)

piaynTaskDataDir = os.path.join(piaynTaskDir, "data")

if not os.path.exists(piaynTaskDataDir):
  os.makedirs(piaynTaskDataDir)    

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [8]:
from datasets import load_dataset
#from Nystromformer.LRA.datasets import text
import pickle, numpy as np

#copy over the custom text.py file to the Nystromformer location
!cp PIAYN/text.py Nystromformer/LRA/datasets

#Create Train test and dev MAP files for the imdb dataset
!python Nystromformer/LRA/datasets/text.py --max_length=$max_length
!cp text.train.pickle /content/drive/MyDrive/PIAYN/text/data
!cp text.dev.pickle /content/drive/MyDrive/PIAYN/text/data
!cp text.test.pickle /content/drive/MyDrive/PIAYN/text/data

max_length:  2000
I0412 05:07:57.208071 140193834309504 dataset_builder.py:811] No config specified, defaulting to first: imdb_reviews/plain_text
2022-04-12 05:07:57.310993: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
I0412 05:07:57.530304 140193834309504 dataset_info.py:434] Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: imdb_reviews/plain_text/1.0.0
I0412 05:07:57.820701 140193834309504 dataset_info.py:361] Load dataset info from /tmp/tmpe4wme_jbtfds
I0412 05:07:57.822672 140193834309504 dataset_info.py:405] Field info.config_name from disk and from code do not match. Keeping the one from code.
I0412 05:07:57.822817 140193834309504 dataset_info.py:405] Fie

In [10]:
from Nystromformer.LRA.code import lra_config
from Nystromformer.LRA.code.dataset import LRADataset
#from Nystromformer.LRA.code.run_tasks import training_config
from torch.utils.data import DataLoader, RandomSampler

#get training config
training_config = lra_config.config[task]["training"]

#Update training config
#training_config["learning_rate"] = 0.00001
training_config["weight_decay"] = 0.1
training_config["eval_frequency"] = 1000

#Check Train Config
print('Training Config: ', training_config)

#get pre-defined model config
model_config = lra_config.config[task]['model']

#Check model Config
print('Model Config: ', model_config)

#Get the dataset
train_dataset = LRADataset(piaynTaskDataDir + f"/{task}.train.pickle", False)
val_dataset = LRADataset(piaynTaskDataDir + f"/{task}.dev.pickle", False)
test_dataset = LRADataset(piaynTaskDataDir + f"/{task}.test.pickle", False)

#Create DataLoader iterators
ds_iter = {
    "train":enumerate(DataLoader(train_dataset, 
                                 #Sample batches randomly for number of specified steps
                                 sampler = RandomSampler(train_dataset, 
                                                         replacement=True, 
                                                         num_samples= training_config["num_train_steps"]*lra_config.config[task]['dataset']['train']), 
                                 batch_size = training_config["batch_size"], 
                                 drop_last = True)),
    "dev":enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True)),
    "test":enumerate(DataLoader(test_dataset, batch_size = 32, drop_last = True)),
}


Training Config:  {'batch_size': 32, 'learning_rate': 0.0001, 'warmup': 8000, 'lr_decay': 'linear', 'weight_decay': 0.1, 'eval_frequency': 1000, 'num_train_steps': 20000, 'num_eval_steps': 781}
Model Config:  {'learn_pos_emb': True, 'tied_weights': False, 'embedding_dim': 64, 'transformer_dim': 64, 'transformer_hidden_dim': 128, 'head_dim': 32, 'num_head': 2, 'num_layers': 2, 'vocab_size': 512, 'max_seq_len': 4000, 'dropout_prob': 0.1, 'attention_dropout': 0.1, 'pooling_mode': 'MEAN', 'num_classes': 2}
Loaded /content/drive/MyDrive/PIAYN/text/data/text.train.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/text.dev.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/text.test.pickle... size=25000


In [13]:
#Check sizes of batches
batch = next((ds_iter['train']))
for k,v in batch[1].items():
  print(k,v.shape)

input_ids_0 torch.Size([32, 2024])
mask_0 torch.Size([32, 2024])
label torch.Size([32])


## Define model

Next, we define our model, and put it on the GPU.

In [14]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
from transformers import PerceiverConfig
#get default perceiver config
configuration = PerceiverConfig()

#Update the Perceiver configurations with Preset model configs
configuration.update(model_config)

#Print Updated Perceiver Configuration
print(configuration)

PerceiverConfig {
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 768,
  "dropout_prob": 0.1,
  "embedding_dim": 64,
  "head_dim": 32,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "learn_pos_emb": true,
  "max_position_embeddings": 2048,
  "max_seq_len": 4000,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_classes": 2,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_head": 2,
  "num_latents": 256,
  "num_layers": 2,
  "num_self_attends_per_block": 26,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "pooling_mode": "MEAN",
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "tied_weights": false,
  "train_size": [
    368,
    496
  ],
  "transformer_dim": 

In [16]:
def initialize_model(config):
  #Initialize Model
  model = PerceiverForSequenceClassification(config)
  
  #Get Model Parameter Counts
  pytorch_total_params = sum(p.numel() for p in model.parameters())
  pytorch_total_params_Trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print('Total Parameters: ', pytorch_total_params, '\nTrainable Parameters: ', pytorch_total_params_Trainable)  

  return model#.to(device)                                          

In [20]:
#Change the perceiver configurations to get total parameters within 10% of BERT
configuration.num_labels = 2
configuration.num_self_attends_per_block = 6
configuration.d_latents = 512
configuration.d_model = 512
configuration.num_layers = 6


model = initialize_model(configuration)

Total Parameters:  14069250 
Trainable Parameters:  14069250


## Train the model

Here we train the model using native PyTorch.

In [18]:
device

device(type='cuda')

In [21]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from datasets import load_metric
import pandas as pd

best_score = 0 
prev_score = 0
maxPatience = 3
currentPatience = 0

#steps = int(training_config["num_train_steps"]/20000)
steps = training_config["num_train_steps"]

optimizer = AdamW(model.parameters(), 
                  lr = training_config["learning_rate"],
                  betas = (0.9, 0.999), 
                  eps = 1e-6, 
                  weight_decay = training_config["weight_decay"])

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer = optimizer,
    max_lr = training_config["learning_rate"],
    pct_start = training_config["warmup"] / training_config["num_train_steps"],
    anneal_strategy = training_config["lr_decay"],
    total_steps = training_config["num_train_steps"]
)

#amp_scaler = torch.cuda.amp.GradScaler() if model_config["mixed_precision"] else None

#initialize training summary
trainingSummary = pd.DataFrame(columns=['step', 'train_acc', 'val_acc'])


model.to(device)

#initialize training accuracy metric
train_accuracy = load_metric("accuracy")

for step in tqdm(range(steps)):  # Perform gradient updates for multiple steps
    
    model.train()
    
    #print("Step:", step)
    #for batch in tqdm(train_dataloader):
    batch = next(ds_iter['train'])[1]

    # get the inputs; 
    inputs = batch["input_ids_0"].to(device)
    attention_mask = batch["mask_0"].to(device)
    labels = batch["label"].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # evaluate
    predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
    accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
    references = batch["label"].numpy()
    train_accuracy.add_batch(predictions=predictions, references=references)
    
    print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

    #delete intermediate variables to free up GPU space
    del loss, outputs, inputs, attention_mask, labels, predictions, accuracy


    #Every 1000 steps validate and save model
    if (step+1)%training_config['eval_frequency']  == 0:
    #if (step+1)%2  == 0:
      
      model.eval()

      val_accuracy = load_metric("accuracy")

      #reset dev iterator
      ds_iter['dev'] = enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True))

      with torch.no_grad():
        for i, batch in tqdm(ds_iter['dev']):
              
          # get the inputs; 
          inputs = batch["input_ids_0"].to(device)
          attention_mask = batch["mask_0"].to(device)
          labels = batch["label"].to(device)

          # forward pass
          outputs = model(inputs=inputs, attention_mask=attention_mask)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          val_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
          del logits, outputs, inputs, attention_mask, labels, predictions, references
      
      #Compute val accuracy
      final_val_score = val_accuracy.compute()['accuracy']
      print("Validation Accuracy:", final_val_score)

      if final_val_score >= best_score:
        best_score = final_val_score
        torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl')
        model.to(device)
      else:
        pass  

      if final_val_score <= prev_score:
        currentPatience += 1
        if currentPatience >= maxPatience:
          print('Patience Limit reached! Stopping early!')
          torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierStep_' + str(step + 1) + 'Token' + str(max_length) + '.pkl')
          break  
      else:
        currentPatience = 0
      
      #Update prev_score
      prev_score = final_val_score

      #Compute training accuracy till now
      final_train_score = train_accuracy.compute()['accuracy']

      #Add to trainingSummary
      trainingSummary.loc[len(trainingSummary.index)] = [step+1, final_train_score, final_val_score]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/20000 [00:00<?, ?it/s]

Loss: 0.7238724231719971, Accuracy: 0.375
Loss: 0.6968759298324585, Accuracy: 0.46875
Loss: 0.6978195905685425, Accuracy: 0.46875
Loss: 0.6912208199501038, Accuracy: 0.53125
Loss: 0.6929813623428345, Accuracy: 0.53125
Loss: 0.6935714483261108, Accuracy: 0.53125
Loss: 0.7163381576538086, Accuracy: 0.46875
Loss: 0.7320137619972229, Accuracy: 0.40625
Loss: 0.7282277941703796, Accuracy: 0.40625
Loss: 0.693403422832489, Accuracy: 0.53125
Loss: 0.7129336595535278, Accuracy: 0.40625
Loss: 0.6990834474563599, Accuracy: 0.40625
Loss: 0.6903167366981506, Accuracy: 0.53125
Loss: 0.6831192374229431, Accuracy: 0.59375
Loss: 0.7038123607635498, Accuracy: 0.4375
Loss: 0.7173801064491272, Accuracy: 0.40625
Loss: 0.7484151124954224, Accuracy: 0.3125
Loss: 0.7253201007843018, Accuracy: 0.40625
Loss: 0.7176125049591064, Accuracy: 0.4375
Loss: 0.7036691904067993, Accuracy: 0.46875
Loss: 0.6855490803718567, Accuracy: 0.5625
Loss: 0.6976220607757568, Accuracy: 0.46875
Loss: 0.6872300505638123, Accuracy: 0.5

0it [00:00, ?it/s]

Validation Accuracy: 0.6181978233034571
Loss: 0.6746785044670105, Accuracy: 0.75
Loss: 0.6812302470207214, Accuracy: 0.5625
Loss: 0.6497948169708252, Accuracy: 0.71875
Loss: 0.6683447360992432, Accuracy: 0.625
Loss: 0.6733297109603882, Accuracy: 0.5
Loss: 0.6505436897277832, Accuracy: 0.65625
Loss: 0.7113781571388245, Accuracy: 0.5
Loss: 0.7182725667953491, Accuracy: 0.40625
Loss: 0.6636832356452942, Accuracy: 0.5625
Loss: 0.64377361536026, Accuracy: 0.65625
Loss: 0.6610810160636902, Accuracy: 0.5625
Loss: 0.6391869783401489, Accuracy: 0.625
Loss: 0.6655495762825012, Accuracy: 0.59375
Loss: 0.7180569171905518, Accuracy: 0.4375
Loss: 0.6792334914207458, Accuracy: 0.59375
Loss: 0.6619933843612671, Accuracy: 0.53125
Loss: 0.6585173606872559, Accuracy: 0.65625
Loss: 0.653066098690033, Accuracy: 0.5625
Loss: 0.6403728127479553, Accuracy: 0.65625
Loss: 0.6762425303459167, Accuracy: 0.71875
Loss: 0.6132648587226868, Accuracy: 0.78125
Loss: 0.6341943144798279, Accuracy: 0.71875
Loss: 0.6025969

...

0it [00:00, ?it/s]

Validation Accuracy: 0.6174775928297055
Loss: 0.46276745200157166, Accuracy: 0.8125
Loss: 0.48762115836143494, Accuracy: 0.75
Loss: 0.6292508840560913, Accuracy: 0.6875
Loss: 0.4473901093006134, Accuracy: 0.78125
Loss: 0.48650220036506653, Accuracy: 0.75
Loss: 0.5950969457626343, Accuracy: 0.6875
Loss: 0.6000405550003052, Accuracy: 0.71875
Loss: 0.4223096966743469, Accuracy: 0.84375
Loss: 0.6503099203109741, Accuracy: 0.6875
Loss: 0.626391589641571, Accuracy: 0.65625
Loss: 0.5124270915985107, Accuracy: 0.75
Loss: 0.4885517656803131, Accuracy: 0.78125
Loss: 0.6134681105613708, Accuracy: 0.6875
Loss: 0.4206119775772095, Accuracy: 0.8125
Loss: 0.4584283232688904, Accuracy: 0.8125
Loss: 0.3991076648235321, Accuracy: 0.84375
Loss: 0.4580909311771393, Accuracy: 0.8125
Loss: 0.3696984350681305, Accuracy: 0.84375
Loss: 0.6238428950309753, Accuracy: 0.625
Loss: 0.5292378664016724, Accuracy: 0.8125
Loss: 0.5795454978942871, Accuracy: 0.71875
Loss: 0.5632175803184509, Accuracy: 0.71875
Loss: 0.41

0it [00:00, ?it/s]

Validation Accuracy: 0.6141965428937259
Patience Limit reached! Stopping early!


TypeError: ignored

In [ ]:
#save training summary
trainingSummary.to_csv(piaynTaskModelDir + '/trainingSummaryToken'+str(max_length)+'.csv')

In [22]:
!nvidia-smi

Tue Apr 12 06:36:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    34W /  70W |   7424MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
del model
torch.cuda.empty_cache()
!nvidia-smi

Tue Apr 12 06:36:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    40W /  70W |   1622MiB / 15109MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [24]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

#load best performing model checkpoint
model = PerceiverForSequenceClassification(configuration)
model.load_state_dict(torch.load(piaynTaskModelDir + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl'))
model.to(device)

model.eval()

with torch.no_grad():
  for i, batch in tqdm(ds_iter['test']):
        
        # get the inputs; 
        inputs = batch["input_ids_0"].to(device)
        attention_mask = batch["mask_0"].to(device)
        labels = batch["label"].to(device)

        # forward pass
        outputs = model(inputs=inputs, attention_mask=attention_mask)
        logits = outputs.logits 
        predictions = logits.argmax(-1).cpu().detach().numpy()
        references = batch["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)

        #delete intermediate variables to free up GPU space
        del logits, outputs, inputs, attention_mask, labels, predictions, references

final_score = accuracy.compute()
print("Accuracy on test set:", final_score['accuracy'])

0it [00:00, ?it/s]

Accuracy on test set: 0.6473271446862996
